In [1]:
from djitellopy import Tello
import cv2
import numpy as np
import time
import math




def detection_center(det):
    cx = (det[3] + det[5]) / 2.0 - 0.5
    cy = (det[4] + det[6]) / 2.0 - 0.5
    return (cx, cy)




def norm(vec):
    return np.sqrt(vec[0]**2 + vec[1]**2)




def closest_detection(detections):
    best_det = None
    min_dist = float('inf')
    for det in detections[0, 0]:
        if det[2] > 0.4 and int(det[1]) == 1:  # 'person'
            center = detection_center(det)
            distance = norm(center)
            if distance < min_dist:
                min_dist = distance
                best_det = det
    return best_det


# def smooth_circle_around_person_test(tello):
#     tello.move_right(4)
#     tello.move_forward(3)
#     tello.rotate_anticlockwise(37)
#     time.sleep(0.2)

# def smooth_circle_around_person(tello, center_x, steps=36, radius=60):
#     """
#     Fly a smooth circle around the detected person.
#     Minimum move length is enforced to avoid SDK 'out of range' errors.
#     """
#     min_step = 20
#     arc_len = max(int(2 * math.pi * radius / steps), min_step)
#     rot_step = int(360 / steps)


#     for i in range(steps):
#         # Optional: recenters horizontally
#         if center_x < 280:
#             tello.move_left(arc_len)
#         elif center_x > 440:
#             tello.move_right(arc_len)


#         tello.move_forward(arc_len)
#         tello.rotate_clockwise(rot_step)
#         time.sleep(0.2)

def curve_circle_around_person(tello, center_x, steps=8, radius=80, speed=30):
    """
    Fly a smooth circular path around the detected person using Tello's curve command.
    """
    angle_step = 2 * math.pi / steps
    z = 0  # Keep constant height relative to current
    
    # Optional: adjust to recenter drone based on detection
    if center_x < 280:
        tello.move_left(20)
    elif center_x > 440:
        tello.move_right(20)
    time.sleep(0.2)

    for i in range(steps):
        theta1 = angle_step * i
        theta2 = angle_step * (i + 0.5)
        theta3 = angle_step * (i + 1)

        # Avoid too-tight turns, enforce min difference between points
        x1 = int(radius * math.cos(theta1))
        y1 = int(radius * math.sin(theta1))
        x2 = int(radius * math.cos(theta2))
        y2 = int(radius * math.sin(theta2))
        x3 = int(radius * math.cos(theta3))
        y3 = int(radius * math.sin(theta3))

        dist1 = math.sqrt((x1 - x2)**2 + (y1 - y2)**2)
        dist2 = math.sqrt((x2 - x3)**2 + (y2 - y3)**2)

        # Skip if distance too small
        if dist1 < 20 or dist2 < 20:
            print(f"Skipping step {i} — move too small.")
            continue

        try:
            tello.curve_xyz_speed(x1, y1, z, x2, y2, z, speed)
        except Exception as e:
            print(f"Curve command failed: {e}")
        time.sleep(0.5)







# Load COCO class names
with open('COCO/object_detection_classes_coco.txt', 'r') as f:
    class_names = f.read().split('\n')


COLORS = np.random.uniform(0, 255, size=(len(class_names), 3))


model = cv2.dnn.readNet(model='COCO/frozen_inference_graph.pb',
                        config='COCO/ssd_mobilenet_v2_coco_2018_03_29.pbtxt.txt',
                        framework='TensorFlow')


tello = Tello()
print("Connecting to Tello...")
tello.connect()
print(f"Battery level: {tello.get_battery()}%")
tello.streamon()


print("Press 't' to takeoff, 'c' to circle person, 'q' to land and quit.")
drone_in_air = False


try:
    while True:
        frame = tello.get_frame_read().frame
        frame = cv2.resize(frame, (720, 480))
        h, w, _ = frame.shape


        blob = cv2.dnn.blobFromImage(frame, size=(300, 300), mean=(104, 117, 123), swapRB=True)
        model.setInput(blob)
        detections = model.forward()
        det = closest_detection(detections)


        if det is not None:
            class_id = int(det[1])
            class_name = class_names[class_id - 1]
            color = COLORS[class_id]


            x1 = int(det[3] * w)
            y1 = int(det[4] * h)
            x2 = int(det[5] * w)
            y2 = int(det[6] * h)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            label = f"{class_name} {det[2]:.2f}"
            cv2.putText(frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)


        cv2.imshow("Tello Tracking Feed", frame)


        key = cv2.waitKey(1) & 0xFF


        if key == ord('t') and not drone_in_air:
            tello.takeoff()
            drone_in_air = True
            print("Drone is now airborne, you can press c to circle or q to land.")
            tello.move_up(40)

        if key == ord('c') and drone_in_air and det is not None:
            center_x = (x1 + x2) // 2
            print("Starting smooth circle around person...")
            curve_circle_around_person(tello, center_x=center_x, steps=8, radius=60, speed=30)
            #smooth_circle_around_person(tello, center_x=center_x, steps=36, radius=30)
            #smooth_circle_around_person_test(tello)


        if key == ord('q'):
            if drone_in_air:
                tello.land()
                print("Drone has landed.")
            break


except Exception as e:
    print(f"Error: {e}")
    if drone_in_air:
        tello.land()

except KeyboardInterrupt:
    print("Interrupted. Landing drone.")
    if drone_in_air:
        tello.land()

finally:
    tello.streamoff()
    cv2.destroyAllWindows()






[INFO] tello.py - 129 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 438 - Send command: 'command'
[INFO] tello.py - 462 - Response command: 'ok'
[INFO] tello.py - 438 - Send command: 'streamon'


Connecting to Tello...
Battery level: 82%


[INFO] tello.py - 462 - Response streamon: 'ok'


Press 't' to takeoff, 'c' to circle person, 'q' to land and quit.


2025-04-19 20:16:11.687 Python[21080:4263558] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.
[INFO] tello.py - 438 - Send command: 'takeoff'
[INFO] tello.py - 462 - Response takeoff: 'ok'
[INFO] tello.py - 438 - Send command: 'up 40'


...

[INFO] tello.py - 438 - Send command: 'curve -60 0 0 -55 -22 0 30'
[INFO] tello.py - 462 - Response curve -60 0 0 -55 -22 0 30: 'error Radius is too large!'
[INFO] tello.py - 438 - Send command: 'curve -60 0 0 -55 -22 0 30'
[INFO] tello.py - 462 - Response curve -60 0 0 -55 -22 0 30: 'error Radius is too large!'
[INFO] tello.py - 438 - Send command: 'curve -60 0 0 -55 -22 0 30'
[INFO] tello.py - 462 - Response curve -60 0 0 -55 -22 0 30: 'error Radius is too large!'


Curve command failed: Command 'curve -60 0 0 -55 -22 0 30' was unsuccessful for 4 tries. Latest response:	'error Radius is too large!'


[INFO] tello.py - 438 - Send command: 'curve -42 -42 0 -22 -55 0 30'
[INFO] tello.py - 462 - Response curve -42 -42 0 -22 -55 0 30: 'error Radius is too large!'
[INFO] tello.py - 438 - Send command: 'curve -42 -42 0 -22 -55 0 30'
[INFO] tello.py - 462 - Response curve -42 -42 0 -22 -55 0 30: 'error Radius is too large!'
[INFO] tello.py - 438 - Send command: 'curve -42 -42 0 -22 -55 0 30'
[INFO] tello.py - 462 - Response curve -42 -42 0 -22 -55 0 30: 'error Radius is too large!'
[INFO] tello.py - 438 - Send command: 'land'


Curve command failed: Command 'curve -42 -42 0 -22 -55 0 30' was unsuccessful for 4 tries. Latest response:	'error Radius is too large!'
Interrupted. Landing drone.


[INFO] tello.py - 462 - Response land: 'ok'
[INFO] tello.py - 438 - Send command: 'streamoff'
[INFO] tello.py - 462 - Response streamoff: 'ok'
